In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
import os
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from lightgbm import LGBMClassifier
import re
import joblib
from lazypredict.Supervised import LazyClassifier


def rmse(y, pred):
    return np.sqrt(mean_squared_error(y, pred))

2022-10-19 16:55:28.492065: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-19 16:55:28.492098: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/competition/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
df = pd.read_csv("data/training_data/preprocessed_180_1.csv")

In [ ]:
len(df)

In [3]:
df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,proton_density,proton_temperature,proton_speed,bx_gsm,by_gsm,bz_gsm,bt,...,In_D,In_I,Jj_X,Jj_Y,Jj_Z,Jj_H,Jj_F,Jj_D,Jj_I,Dindex
0,0,0,89.50,5.97,63349.11,374.65,-1.19,4.71,-1.53,5.35,...,-8.18,53.53,33215.99,-286.17,31620.31,33217.22,45860.96,-0.49,43.59,3.00
1,1,1,269.50,5.99,59161.81,370.63,-2.47,4.95,-2.63,6.56,...,-8.23,53.53,33209.00,-313.60,31614.71,33210.48,45852.22,-0.54,43.59,3.00
2,2,2,449.50,5.62,69938.55,377.34,-3.34,5.26,1.57,7.64,...,-8.23,53.53,33210.12,-309.23,31621.87,33211.56,45857.95,-0.53,43.60,1.00
3,3,3,629.50,5.61,83242.99,403.80,2.07,7.53,-6.16,11.85,...,-8.21,53.54,33211.10,-303.70,31622.53,33212.49,45859.06,-0.52,43.60,3.00
4,4,4,809.50,5.70,170966.78,445.52,-1.93,8.20,-2.96,13.91,...,-8.21,53.58,33161.93,-302.83,31623.31,33163.32,45824.00,-0.52,43.64,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23371,23371,23371,4206869.50,6.58,89799.83,438.73,1.41,2.39,2.02,4.05,...,-7.94,53.79,33215.36,-652.36,31963.44,33221.76,46101.49,-1.13,43.89,1.00
23372,23372,23372,4207049.50,6.54,87216.37,427.42,-0.70,2.13,-1.97,3.98,...,-7.94,53.79,33216.24,-652.49,31964.94,33222.64,46103.16,-1.13,43.89,2.00
23373,23373,23373,4207229.50,8.22,76142.33,429.93,0.95,4.34,1.72,5.40,...,-7.94,53.78,33217.10,-652.82,31966.49,33223.51,46104.87,-1.13,43.90,3.00
23374,23374,23374,4207409.50,9.48,93696.57,429.54,1.52,2.38,3.50,5.16,...,-7.95,53.78,33223.42,-657.47,31967.08,33229.93,46109.90,-1.13,43.89,1.00


In [14]:
# df['datetime'] = df.datetime.apply(pd.to_datetime)

def evaluate_regr(y, pred):
    rmse_val = rmse(y, pred)
    print('RMSE: ', rmse_val)

y_target = df['Dindex']
X_features = df.drop(['Dindex', 'Unnamed: 0','Unnamed: 0.1', 'Unnamed: 0.2'], axis=1, inplace=False)
input_var = ['proton_density', 'proton_speed', 'proton_temperature', 'bx_gsm', 'by_gsm', 'bz_gsm', 'bt', 
            'gn_X', 'gn_Y', 'gn_Z', 'gn_H', 'gn_F', 'gn_D', 'gn_I', 'In_X', 'Jj_X', 'Jj_Y', 'Jj_Z', 'Jj_F', 'Jj_I']
input_X = X_features[input_var]
# X_train, X_test, y_train, y_test = train_test_split(input_X, y_target, test_size=0.1, random_state=1)
X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.1, random_state=1)


In [23]:
submission = pd.read_csv('data/quiz/preprocessed.csv')
submission = submission.drop(['Unnamed: 0'], axis=1, inplace=False)
submission = submission[input_var]
pred = xgbc.predict(submission)
submission['Dindex'] = pred
submission[240:]['Dindex'].to_csv("data/submit.csv", index = False)

In [112]:
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
def evaluate_regr(y, pred):
    rmse_val = rmse(y, pred)
    print('RMSE: ', rmse_val)

#y_data = df.pop('Dindex')
#x_data = df
y_target = df['Dindex']
X_features = df.drop(['Dindex', 'Unnamed0', 'Unnamed02', 'Unnamed01'], axis=1, inplace=False)

X_train, X_test, y_train, y_test = train_test_split(X_features, y_target, test_size=0.01, random_state=0)

lr_reg = LGBMClassifier(n_estimators=500)
lr_reg.fit(X_train, y_train)
pred = lr_reg.predict(X_test)

evaluate_regr(y_test, pred)
# joblib.dump(lr_reg, 'lgb.pkl')


RMSE:  1.0621700090875887


In [9]:
# gbm_pickle = joblib.load('lgb.pkl')
submission = pd.read_csv('data/quiz/preprocessed.csv')
submission = submission.drop(['Unnamed: 0'], axis=1, inplace=False)
pred = lr_reg.predict(submission)
submission['Dindex'] = pred
submission.to_csv("data/submit.csv", index = False)


In [59]:
print(submission)

     proton_density  proton_temperature  proton_speed    bx_gsm    by_gsm  \
0          9.740889       425579.922222    512.838889  2.062833 -2.555000   
1          9.482889       442720.550000    508.746111 -2.598778  0.613778   
2          8.170778       379750.555556    500.266111 -2.452833  2.536000   
3          9.392056       401559.461111    508.778333 -1.083333  3.821389   
4          6.388306       264431.102778    483.449167 -2.386889  3.520472   
..              ...                 ...           ...       ...       ...   
715        5.114333       234353.788889    466.514444  5.074000 -2.140111   
716        7.889639       283785.755556    505.847500  3.460778 -1.112556   
717        7.943278       248275.350000    506.611667  2.695278 -2.575889   
718        6.976667       246099.133333    503.444444  3.394111 -3.976056   
719        6.443056       218375.266667    494.004444  3.749167 -4.214556   

       bz_gsm        bt          gn_X        gn_Y          gn_Z  ...  \
0  

In [33]:
print(X_features)

       proton_density  proton_temperature  proton_speed    bx_gsm    by_gsm  \
0            5.969653        63349.109764    374.652888 -1.191787  4.710839   
1            5.993983        59161.813942    370.626311 -2.469191  4.945273   
2            5.618350        69938.545719    377.336518 -3.337570  5.263640   
3            5.614182        83242.987856    403.796592  2.066158  7.528629   
4            5.704158       170966.782678    445.517097 -1.931976  8.198496   
...               ...                 ...           ...       ...       ...   
23371        6.578556        89799.827778    438.730556  1.414111  2.388500   
23372        6.542222        87216.372222    427.420000 -0.699833  2.133722   
23373        8.224667        76142.327778    429.934444  0.952167  4.338000   
23374        9.480444        93696.566667    429.536111  1.519444  2.384778   
23375        5.804972        80350.052778    404.253611 -2.070778  2.120722   

         bz_gsm         bt          gn_X        gn_

In [15]:
clf = LazyClassifier(verbose=0, predictions=True)

models, predictions = clf.fit(X_train, X_test, y_train, y_test)

models

 90%|████████▉ | 26/29 [01:57<00:21,  7.01s/it]

[07:45:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


100%|██████████| 29/29 [02:05<00:00,  4.33s/it]


,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,0.23,0.43,None,0.23,0.02
LinearDiscriminantAnalysis,0.49,0.40,None,0.43,0.04
GaussianNB,0.41,0.39,None,0.40,0.02
AdaBoostClassifier,0.27,0.36,None,0.27,1.54
ExtraTreesClassifier,0.51,0.35,None,0.49,1.87
XGBClassifier,0.50,0.34,None,0.48,7.76
RandomForestClassifier,0.51,0.34,None,0.49,5.52
BaggingClassifier,0.46,0.34,None,0.45,1.82
DecisionTreeClassifier,0.40,0.32,None,0.40,0.32


In [21]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


#0.7250 ,1500, 0.1
xgbc = XGBClassifier(
    learning_rate=0.05,
    n_estimators=700,
    max_depth=5,
    min_child_weight=5,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    nthread=-1,
    seed=2019
    )
xgbc.fit(X_train, y_train)
pred = xgbc.predict(X_test)

evaluate_regr(y_test, pred)

[07:52:20] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RMSE:  0.7396002616336389


In [22]:
submission = pd.read_csv('data/quiz/preprocessed.csv')
submission = submission.drop(['Unnamed: 0'], axis=1, inplace=False)
pred = xgbc.predict(submission)
submission['Dindex'] = pred
submission[240:]['Dindex'].to_csv("data/submit.csv", index = False)

ValueError: Feature shape mismatch, expected: 11, got 28

In [16]:
X_features

,proton_density,proton_temperature,proton_speed,bx_gsm,by_gsm,bz_gsm,bt,gn_X,gn_Y,gn_Z,...,In_F,In_D,In_I,Jj_X,Jj_Y,Jj_Z,Jj_H,Jj_F,Jj_D,Jj_I
0,5.97,63349.11,374.65,-1.19,4.71,-1.53,5.35,29632.76,-111.92,41079.20,...,50595.45,-8.18,53.53,33215.99,-286.17,31620.31,33217.22,45860.96,-0.49,43.59
1,5.99,59161.81,370.63,-2.47,4.95,-2.63,6.56,29628.98,-137.78,41075.50,...,50587.19,-8.23,53.53,33209.00,-313.60,31614.71,33210.48,45852.22,-0.54,43.59
2,5.62,69938.55,377.34,-3.34,5.26,1.57,7.64,29632.27,-134.26,41078.96,...,50592.18,-8.23,53.53,33210.12,-309.23,31621.87,33211.56,45857.95,-0.53,43.60
3,5.61,83242.99,403.80,2.07,7.53,-6.16,11.85,29632.70,-126.09,41080.14,...,50593.85,-8.21,53.54,33211.10,-303.70,31622.53,33212.49,45859.06,-0.52,43.60
4,5.70,170966.78,445.52,-1.93,8.20,-2.96,13.91,29585.53,-124.44,41082.73,...,50568.52,-8.21,53.58,33161.93,-302.83,31623.31,33163.32,45824.00,-0.52,43.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23371,6.58,89799.83,438.73,1.41,2.39,2.02,4.05,29628.33,-458.37,41412.76,...,50872.34,-7.94,53.79,33215.36,-652.36,31963.44,33221.76,46101.49,-1.13,43.89
23372,6.54,87216.37,427.42,-0.70,2.13,-1.97,3.98,29628.82,-458.08,41412.40,...,50872.96,-7.94,53.79,33216.24,-652.49,31964.94,33222.64,46103.16,-1.13,43.89
23373,8.22,76142.33,429.93,0.95,4.34,1.72,5.40,29629.86,-458.85,41413.11,...,50873.85,-7.94,53.78,33217.10,-652.82,31966.49,33223.51,46104.87,-1.13,43.90
23374,9.48,93696.57,429.54,1.52,2.38,3.50,5.16,29636.65,-463.62,41413.73,...,50878.49,-7.95,53.78,33223.42,-657.47,31967.08,33229.93,46109.90,-1.13,43.89


In [128]:
xgbc = XGBClassifier(
    learning_rate =0.1,
    n_estimators=800,
    max_depth=4,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective= 'binary:logistic',
    nthread=-1,
    seed=2019
    )

In [18]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost
classifier = xgboost.XGBClassifier()


NameError: name 'gpu_hist' is not defined

In [19]:
import xgboost as xgb
from xgboost import XGBClassifier
xgb.__version__

'1.5.0'

In [7]:
from sklearn.model_selection import GridSearchCV

#0.7215 ,1500
# learning_rate=0.1,
#     n_estimators=3000,
#     max_depth=4,
#     min_child_weight=1,
#     gamma=0,
#     subsample=0.8,
#     colsample_bytree=0.8,
#     nthread=-1,
#     seed=2019
gbm = XGBClassifier()
param_grid = {
    'learning_rate': [0.01,0.05,0.1,0.15],
    'n_estimators': [1500],
    'max_depth': [4],
    'min_child_weight': [1],
    'gamma': [0],
    'subsample': [0.8],
    'colsample_bytree': [0.8],
    'nthread': [-1],
    'seed':[2019]
}
grid_mse = GridSearchCV(param_grid=param_grid, estimator=gbm, 
                        scoring='neg_root_mean_squared_error', cv=2, verbose=1)

grid_mse.fit(X_train, y_train)

print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[03:37:55] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


KeyboardInterrupt: 

In [14]:
max_depth_lists = [3,4,5,6,7,8]
min_child_weight_lists = [1,2,3,4,5,6]
gamma_lists = [0,0.1,0.2,0.3,0.4,0.5]
subsample_lists = [0.6,0.7,0.8,0.9,1.0]
subsample_lists2 = [i/100.0 for i in range(71,90)]
colsample_bytree_lists = [0.6,0.7,0.8,0.9,1.0]
reg_alpha_lists = [1e-5,1e-4,1e-3,1e-2,0.1,1,10,100]
n_estimator_lists = [i for i in range(100,6000,100)]
learning_rate_lists = [0.05, 0.1,0.15,0.20,0.25,0.3]
# for gamma_list in gamma_lists:
    #0.7250 ,1500, 0.1
    #1500, 0.1
for subsample_list in subsample_lists:
    for colsample_bytree_list in colsample_bytree_lists:
# for reg_alpha_list in reg_alpha_lists:
# for n_estimator_list in n_estimator_lists:
#     for learning_rate_list in learning_rate_lists:

# for max_depth_list in max_depth_lists:
#     for min_child_weight_list in min_child_weight_lists:
        
        xgbc = XGBClassifier(
        learning_rate=0.05,
        n_estimators=700,
        max_depth=5,
        min_child_weight=5,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        nthread=-1,
        seed=2019
        )
        xgbc.fit(X_train, y_train)
        pred = xgbc.predict(X_test)

        evaluate_regr(y_test, pred)
#         print('gamma =', gamma_list)
        print('subsample =', subsample_list, 'colsample_bytree_list =', colsample_bytree_list)
    #     print('subsample =', subsample_list)
    #     print('reg_alpha =', reg_alpha_list)
#         print('n_estimator =', n_estimator_list, 'learning_rate =', learning_rate_list)
#         print('max_depth=', max_depth_list, 'min_child_weight=', min_child_weight_list)
        

[04:41:39] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RMSE:  0.7704997100912119
subsample = 0.6 colsample_bytree_list = 0.6
[04:42:09] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RMSE:  0.772440169350798
subsample = 0.6 colsample_bytree_list = 0.7
[04:42:39] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
RMSE:  0.7801537432653138
subsample = 0.6 colsample_bytree_list = 0.8
[04:43:09] WARNING: ../src/learn

In [138]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold

# max_depth_lists = [i for i in range(1,10)]
min_samples_leaf_lists = [i for i in range(1,15)]
# min_child_weight_lists = [1,2,3,4,5,6]
# gamma_lists = [0,0.1,0.2,0.3,0.4,0.5]
# subsample_lists = [0.6,0.7,0.8,0.9,1.0]
# subsample_lists2 = [i/100.0 for i in range(71,90)]
# colsample_bytree_lists = [0.6,0.7,0.8,0.9,1.0]
# reg_alpha_lists = [1e-5,1e-4,1e-3,1e-2,0.1,1,10,100]
# n_estimators_lists = [i for i in range(100,6000,100)]
# learning_rate_lists = [0.05, 0.1,0.15,0.20,0.25,0.3]

# for n_estimators in n_estimators_lists:
for min_samples_leaf in min_samples_leaf_lists:
    for i in range(3,6):
        x = LinearDiscriminantAnalysis(
            min_samples_leaf=min_samples_leaf
            )
        x.fit(X_train, y_train)
        pred = x.predict(X_test)
        evaluate_regr(y_test, pred)
#         print('n_estimators =', n_estimators)
        print('min_samples_leaf =', min_samples_leaf)


RMSE:  0.8118561145468557
min_samples_leaf = 1
RMSE:  0.8163219479279898
min_samples_leaf = 1
RMSE:  0.8100100855860187
min_samples_leaf = 1
RMSE:  0.7988443320790239
min_samples_leaf = 2
RMSE:  0.8017837257372732
min_samples_leaf = 2
RMSE:  0.7940109438517664
min_samples_leaf = 2
RMSE:  0.795356498017879
min_samples_leaf = 3
RMSE:  0.8015169535702689
min_samples_leaf = 3
RMSE:  0.793472083291666
min_samples_leaf = 3
RMSE:  0.8071006522043631
min_samples_leaf = 4
RMSE:  0.7999144522266709
min_samples_leaf = 4
RMSE:  0.8049780958610502
min_samples_leaf = 4
RMSE:  0.8097460236114106
min_samples_leaf = 5
RMSE:  0.8028499279712591
min_samples_leaf = 5
RMSE:  0.811329106302714
min_samples_leaf = 5
RMSE:  0.8173691862648487
min_samples_leaf = 6
RMSE:  0.8244032499710771
min_samples_leaf = 6
RMSE:  0.8178923025976081
min_samples_leaf = 6
RMSE:  0.8205028815392424
min_samples_leaf = 7
RMSE:  0.8288014783437014
min_samples_leaf = 7
RMSE:  0.8210240012370873
min_samples_leaf = 7
RMSE:  0.8311205

In [17]:
X_train, X_test, y_train, y_test = train_test_split(input_X, y_target, test_size=0.001, random_state=1)

In [18]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import RidgeClassifierCV
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier
from sklearn.dummy import DummyClassifier


# 900 600 1600 2000
y = LinearDiscriminantAnalysis()
y.fit(X_train, y_train)
pred = y.predict(X_test)
evaluate_regr(y_test, pred)

RMSE:  0.7637626158259734


In [19]:
submission = pd.read_csv('data/quiz/preprocessed.csv')
# submission = submission.drop(['Unnamed: 0'], axis=1, inplace=False)
submission = submission[input_var]
pred = y.predict(submission)
submission['Dindex'] = pred
submission[240:]['Dindex'].to_csv("data/submit.csv", index = False)